In [1]:
# Setup imports and logging
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import datetime

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# Import data
COLUMNS = ["media_height", "media_width", "media_type", "media_waste", "mode_name", "print_height",
           "print_width","print_id", "R", "G", "B","width","height","w/h","size","print_time_sec"]
FEATURES = ["media_height", "media_width", "media_type", "media_waste", "mode_name", "print_height",
           "print_width","print_id", "R", "G", "B","width","height","w/h","size"]
LABEL = "print_time_sec"

training_set = pd.read_csv("data_sheet.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("data_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("data_valid.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

In [4]:
# Define feature columns
feature_cols = [tf.contrib.layers.real_valued_column(k)
                for k in FEATURES]

In [5]:
# Define regressor
'''
regressor = tf.contrib.learn.DNNRegressor(
    feature_columns=feature_cols,
    hidden_units=[15, 15],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l2_regularization_strength=0.5),
    model_dir="time_model")
'''
# Define regressor
regressor = tf.contrib.learn.DNNRegressor(
    feature_columns=feature_cols,
    hidden_units=[10, 10],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.0001,
      l1_regularization_strength=10.0,
      l2_regularization_strength=10.0
    ),
    model_dir='time_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b6a652950>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [6]:
# Define input_fn
def input_fn(data_set):
    #data_set = (data_set - data_set.mean()) / (data_set.max() - data_set.min())
    feature_cols = {
        k: tf.constant(
            data_set[k].values,
            # NOTE: define shape explicitly to get rid of the warning
            # Input to shape should be a list
            shape=[data_set[k].size, 1]
        ) for k in FEATURES
    }
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

def output_fn(data_set):
    #data_set = (data_set - data_set.mean()) / (data_set.max() - data_set.min())
    feature_cols = {
        k: tf.constant(
            data_set[k].values,
            # NOTE: define shape explicitly to get rid of the warning
            # Input to shape should be a list
            shape=[data_set[k].size, 1]
        ) for k in FEATURES
    }
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

In [ ]:
#input_fn(prediction_set)
#prediction_set = (prediction_set - prediction_set.mean()) / (prediction_set.max() - prediction_set.min())
#prediction_set

In [7]:
# Train the model
t1 = datetime.datetime.now()
regressor.fit(input_fn=lambda: input_fn(training_set), steps=600)

t2 = datetime.datetime.now()

print (t2-t1)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from time_model/model.ckpt-27768774


InvalidArgumentError: Nan in summary histogram for: dnn/dnn/hiddenlayer_0_activation
	 [[Node: dnn/dnn/hiddenlayer_0_activation = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](dnn/dnn/hiddenlayer_0_activation/tag, dnn/hiddenlayer_0/hiddenlayer_0/Relu)]]

Caused by op u'dnn/dnn/hiddenlayer_0_activation', defined at:
  File "/home/hao/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/hao/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/hao/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hao/anaconda2/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hao/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelap

In [ ]:
# Evaluate the model
ev = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)

loss_score = ev['loss']
print('Loss: {0:f}'.format(loss_score))

In [ ]:
# Make Predictions
y = regressor.predict(input_fn=lambda: output_fn(prediction_set))
predictions = list(itertools.islice(y, 10))
print('Predictions: {}'.format(str(predictions)))

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [ ]:
# 加载数据集
X = train_x.values
y = train_y.values
nn = SKCompat(nn)
# 用SVM进行学习并记录loss
train_sizes, train_loss, test_loss = learning_curve(nn(params=model_params), 
                                                    X, y, cv = 10, scoring = 'neg_mean_squared_error',
                                                    train_sizes = [0.1, 0.25, 0.5, 0.75, 1])
# 训练误差均值
train_loss_mean = -np.mean(train_loss, axis = 1)
# 测试误差均值
test_loss_mean = -np.mean(test_loss, axis = 1)
# 绘制误差曲线
plt.plot(train_sizes, train_loss_mean, 'o-', color = 'r', label = 'Training')
plt.plot(train_sizes, test_loss_mean, 'o-', color = 'g', label = 'Cross-Validation')
plt.xlabel('Training data size')
plt.ylabel('Loss')
plt.legend(loc = 'best')
plt.show()